# Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
from tqdm import tqdm
from astropy.visualization import simple_norm
import pickle

from scipy.stats import stats
import math
import os

import datetime
datecode = '{}-{}-{}'.format(datetime.datetime.now().year, datetime.datetime.now().month, datetime.datetime.now().day)

interim_dir = '/mnt/c/Projects/Blogs/AusAEM_blog_TS/data/interim/'
processed_dir = '/mnt/c/Projects/Blogs/AusAEM_blog_TS/data/processed/'
raw_dir = '/mnt/c/Projects/Blogs/AusAEM_blog_TS/data/raw/'
external_dir = '/mnt/c/Projects/Blogs/AusAEM_blog_TS/data/external/'

# make new folders in interim and processed directories for today's date

if os.path.join(interim_dir, datecode) not in [x[0] for x in os.walk(interim_dir)]:
    os.mkdir(os.path.join(interim_dir, datecode))
if os.path.join(processed_dir, datecode) not in [x[0] for x in os.walk(processed_dir)]:
    os.mkdir(os.path.join(processed_dir, datecode))

print(datecode)



2022-6-29


## Custom functions

In [6]:
# Function to extract column headers from dfn file via Mnyikka https://stackoverflow.com/questions/3368969/find-string-between-two-substrings

def GetListOfSubstrings(stringSubject,string1,string2):
    MyList = []
    intstart=0
    strlength=len(stringSubject)
    continueloop = 1

    while(intstart < strlength and continueloop == 1):
        intindex1=stringSubject.find(string1,intstart)
        if(intindex1 != -1): #The substring was found, lets proceed
            intindex1 = intindex1+len(string1)
            intindex2 = stringSubject.find(string2,intindex1)
            if(intindex2 != -1):
                subsequence=stringSubject[intindex1:intindex2]
                MyList.append(subsequence)
                intstart=intindex2+len(string2)
            else:
                continueloop=0
        else:
            continueloop=0
    return MyList

def interp_along_axis(y, x, newx, axis=1, inverse=False, method='linear'):
    """ Interpolate vertical profiles, e.g. of atmospheric variables
    using vectorized numpy operations

    This function assumes that the x-xoordinate increases monotonically

    #### EDIT BY ####
    Thomas Schaap
    February 2022
    * Updated to take 1Darray for input x and newx
    * Updated to 'axis' default = 1, probably breaks if specified otherwise

    ps:
    * Updated to work with irregularly spaced x-coordinate.
    * Updated to work with irregularly spaced newx-coordinate
    * Updated to easily inverse the direction of the x-coordinate
    * Updated to fill with nans outside extrapolation range
    * Updated to include a linear interpolation method as well
        (it was initially written for a cubic function)

    Peter Kalverla
    March 2018

    --------------------
    More info:
    Algorithm from: http://www.paulinternet.nl/?page=bicubic
    It approximates y = f(x) = ax^3 + bx^2 + cx + d
    where y may be an ndarray input vector
    Returns f(newx)

    The algorithm uses the derivative f'(x) = 3ax^2 + 2bx + c
    and uses the fact that:
    f(0) = d
    f(1) = a + b + c + d
    f'(0) = c
    f'(1) = 3a + 2b + c

    Rewriting this yields expressions for a, b, c, d:
    a = 2f(0) - 2f(1) + f'(0) + f'(1)
    b = -3f(0) + 3f(1) - 2f'(0) - f'(1)
    c = f'(0)
    d = f(0)

    These can be evaluated at two neighbouring points in x and
    as such constitute the piecewise cubic interpolator.
    """

    x = np.array([np.array(x) for i in np.empty(y.shape[0])])

    newx = np.array([np.array(newx) for i in np.empty(y.shape[0])])

    # View of x and y with axis as first dimension
    if inverse:
        _x = np.moveaxis(x, axis, 0)[::-1, ...]
        _y = np.moveaxis(y, axis, 0)[::-1, ...]
        _newx = np.moveaxis(newx, axis, 0)[::-1, ...]
    else:
        _y = np.moveaxis(y, axis, 0)
        _x = np.moveaxis(x, axis, 0)
        _newx = np.moveaxis(newx, axis, 0)

    if np.any(np.diff(_x, axis=0) < 0):
        raise ValueError('x should increase monotonically')
    if np.any(np.diff(_newx, axis=0) < 0):
        raise ValueError('newx should increase monotonically')

    # Cubic interpolation needs the gradient of y in addition to its values
    if method == 'cubic':
        # For now, simply use a numpy function to get the derivatives
        # This produces the largest memory overhead of the function and
        # could alternatively be done in passing.
        ydx = np.gradient(_y, axis=0, edge_order=2)

    # This will later be concatenated with a dynamic '0th' index
    ind = [i for i in np.indices(_y.shape[1:])]

    # Allocate the output array
    original_dims = _y.shape
    newdims = list(original_dims)
    newdims[0] = len(_newx)
    newy = np.zeros(newdims)

    # set initial bounds
    i_lower = np.zeros(_x.shape[1:], dtype=int)
    i_upper = np.ones(_x.shape[1:], dtype=int)
    x_lower = _x[0, ...]
    x_upper = _x[1, ...]

    for i, xi in enumerate(_newx):
        # Start at the 'bottom' of the array and work upwards
        # This only works if x and newx increase monotonically

        # Update bounds where necessary and possible
        needs_update = (xi > x_upper) & (i_upper+1<len(_x))
        # print x_upper.max(), np.any(needs_update)
        while np.any(needs_update):
            i_lower = np.where(needs_update, i_lower+1, i_lower)
            i_upper = i_lower + 1
            x_lower = _x[[i_lower]+ind]
            x_upper = _x[[i_upper]+ind]

            # Check again
            needs_update = (xi > x_upper) & (i_upper+1<len(_x))

        # Express the position of xi relative to its neighbours
        xj = (xi-x_lower)/(x_upper - x_lower)

        # Determine where there is a valid interpolation range
        within_bounds = (_x[0, ...] < xi) & (xi < _x[-1, ...])

        if method == 'linear':
            f0, f1 = _y[[i_lower]+ind], _y[[i_upper]+ind]
            a = f1 - f0
            b = f0

            newy[i, ...] = np.where(within_bounds, a*xj+b, np.nan)

        elif method=='cubic':
            f0, f1 = _y[[i_lower]+ind], _y[[i_upper]+ind]
            df0, df1 = ydx[[i_lower]+ind], ydx[[i_upper]+ind]

            a = 2*f0 - 2*f1 + df0 + df1
            b = -3*f0 + 3*f1 - 2*df0 - df1
            c = df0
            d = f0

            newy[i, ...] = np.where(within_bounds, a*xj**3 + b*xj**2 + c*xj + d, np.nan)

        else:
            raise ValueError("invalid interpolation method"
                             "(choose 'linear' or 'cubic')")

    if inverse:
        newy = newy[::-1, ...]

    return np.moveaxis(newy, 0, axis)

# Data in

In [3]:
QLD_EM_dat_fn = '/mnt/c/Projects/Blogs/AusAEM_blog_TS/data/raw/Final_QLD_Regional_lines/located_data/AusAEM_Year1_QLD_Final_EM.dat'
QLD_EM_dfn_fn = '/mnt/c/Projects/Blogs/AusAEM_blog_TS/data/raw/Final_QLD_Regional_lines/located_data/AusAEM_Year1_QLD_Final_EM.dfn'

with open(QLD_EM_dfn_fn, 'r') as file:
    QLD_EM_dfn = file.read()

column_names = GetListOfSubstrings(QLD_EM_dfn, 'RT=;', ':')
column_names_list = []
for header in column_names:
    if header.startswith('EM'):
        column_names_list = column_names_list + [header + '[{}]'.format(i) for i in range(1,16)]
    else:
        column_names_list.append(header)

EMZ_HPRG_list = ['EMZ_HPRG' + '[{}]'.format(i) for i in range(1,16)]

QLD_EM = pd.read_csv(QLD_EM_dat_fn, header = None, delim_whitespace=True, nrows = 1)
QLD_EM.columns = column_names_list

select_cols = ['Line', 'Easting', 'Northing'] + EMZ_HPRG_list
col_index = []
for item in select_cols:
    col_index.append(QLD_EM.columns.get_loc(item))

QLD_EM = pd.read_csv(QLD_EM_dat_fn, header = None, delim_whitespace=True, usecols = col_index, na_values=-999.999999)

QLD_EM.columns = select_cols
QLD_EM = QLD_EM.dropna()

# Line distance calculation, resampling

In [4]:
# Create along-line distance field

append_here = np.array([])
for line in tqdm(QLD_EM.Line.unique()):
    data = QLD_EM[QLD_EM.Line == line].sort_values('Easting')
    array = np.zeros_like(data)
    array = array[:,0]
    for i in range(len(data)):
        if i == 0:
            array[i] = 0
        else:
            array[i] = array[i-1] + np.sqrt(np.abs((data['Easting'].iloc[i] - data['Easting'].iloc[i-1]))**2 + np.abs((data['Northing'].iloc[i] - data['Northing'].iloc[i-1]))**2)
    append_here=  np.append(append_here, array)
QLD_EM['line_distance'] = append_here

100%|██████████| 159/159 [01:23<00:00,  1.90it/s]


In [7]:
# Resample data to consistent 20 m interval

x_interval = 20 # m
x_interp = np.array([])
EMZ_HPRG_interp = []
E_interp = []
N_interp = []
line_list = np.array([])
for line in tqdm(QLD_EM.Line.unique()):
    data = QLD_EM[QLD_EM.Line == line]
    x = QLD_EM[QLD_EM.Line == line]['line_distance']
    y = data[EMZ_HPRG_list].values.T
    x = data['line_distance']
    E = data['Easting']
    N = data['Northing']
    newx = np.arange(0,x.max(), x_interval)
    line_ar = np.full(newx.shape, line)
    line_list = np.append(line_list, line_ar)

    x_interp = np.append(x_interp, newx)
    new_E = np.interp(newx, x, E)
    E_interp.append(new_E)

    new_N = np.interp(newx, x, N)
    N_interp.append(new_N)

    new_EMZ_HPRG = interp_along_axis(y, x, newx)
    EMZ_HPRG_interp.append(new_EMZ_HPRG)

EMZ_HPRG_interp_ar = np.concatenate(EMZ_HPRG_interp, axis =1)
new_E_ar = np.concatenate(E_interp)
new_N_ar = np.concatenate(N_interp)

  0%|          | 0/159 [00:00<?, ?it/s]/home/taschaap/miniconda3/envs/datascience_37/lib/python3.7/site-packages/ipykernel_launcher.py:135: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/taschaap/miniconda3/envs/datascience_37/lib/python3.7/site-packages/ipykernel_launcher.py:122: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/taschaap/miniconda3/envs/datascience_37/lib/python3.7/site-packages/ipykernel_launcher.py:123: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` inste

# Output

In [9]:
# Generate dataframe, save csv

QLD_interp = pd.DataFrame()
QLD_interp['x'] = x_interp
QLD_interp['Line'] = line_list.astype(int)
QLD_interp['E'] = new_E_ar
QLD_interp['N'] = new_N_ar
QLD_interp[EMZ_HPRG_list] = pd.DataFrame(EMZ_HPRG_interp_ar.T)
QLD_interp = QLD_interp.dropna()

QLD_interp.to_csv('/mnt/c/Projects/Blogs/AusAEM_blog_TS/data/processed/QLD_AusEM_interp.csv', index = False)